In [ ]:
import os
import json
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt

import torch
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader

In [ ]:
class config:
    dev_images_path = '/kaggle/input/vlsp-dataset/dev-images/dev-images'
    dev_json_path = '/kaggle/input/vlsp-dataset/vlsp2023_dev_data.json'
    train_images_path = '/kaggle/input/vlsp-dataset/training-data-20230913T111851Z-001/training-data/training-images/training-images'
    train_json_path = '/kaggle/input/vlsp-dataset/training-data-20230913T111851Z-001/training-data/vlsp2023_train_data.json'
#     model_dir = ''
#     TRAIN_SIZE = 
#     SEED = 
#     EPOCHS = 
#     MAX_LEN = 
#     TRAIN_BATCH_SIZE = 
#     VAL_BATCH_SIZE = 
#     target_list = []
#     NUM_WORKERS = os.cpu_count()
    
# torch.manual_seed(config.SEED)
# torch.cuda.manual_seed(config.SEED)
# device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# print(device)

In [ ]:
print("length dev-images:", len([name for name in os.listdir(config.dev_images_path) if os.path.isfile(os.path.join(config.dev_images_path, name))]))
print("length train-images:", len([name for name in os.listdir(config.train_images_path) if os.path.isfile(os.path.join(config.train_images_path, name))]))

In [ ]:
class VLSP_Dataset(Dataset):
    def __init__(self, json_file, image_dir, transform=None):
        self.image_dir = image_dir
        self.transform = transform

        with open(json_file, 'r') as json_file:
            data = json.load(json_file)

        self.dataset = []

        for annotation_id, annotation_data in data['annotations'].items():
            image_id = annotation_data['image_id']
            question = annotation_data['question']
            answer = annotation_data['answer']

            image_filename = data['images'][str(image_id)]
            image_path = f'{image_dir}/{image_filename}'
            label = f"{question} [SEP] {answer}"

            self.dataset.append((image_path, label))

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        image_path, label = self.dataset[idx]
        image = Image.open(image_path).convert('RGB')

        if self.transform:
            image = self.transform(image)

        return image, label

In [ ]:
dev_json_file = config.dev_json_path
dev_image_dir = config.dev_images_path

train_json_file = config.train_json_path
train_image_dir = config.train_images_path

vlsp_dataset = VLSP_Dataset(dev_json_file, dev_image_dir, transform=transforms.Compose([transforms.Resize((224, 224)), transforms.ToTensor()]))

In [ ]:
random_indices = np.random.choice(len(vlsp_dataset), 5)

for idx in random_indices:
    image, label = vlsp_dataset[idx]
    
    image = image.permute(1, 2, 0).numpy()
    
    plt.figure(figsize=(8, 8))
    plt.imshow(image)
    plt.title(f"Label: {label}")
    plt.axis('off')
    plt.show()